In [1]:
import matplotlib.pyplot as plt
import numpy as np
from IPython import display
import time
from matplotlib.patches import Ellipse
import requests
from bs4 import BeautifulSoup
import json 
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from jupyter_ui_poll import ui_events

In [2]:
dot_size = 100
blue_color = 'blue'
yellow_color = 'orange'
black_color = 'black'

In [3]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return
    
btn1 = widgets.Button(description="Left")
btn2 = widgets.Button(description="Right")

btn1.on_click(register_event) 
btn2.on_click(register_event) 


In [4]:
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


In [5]:
def generate_dots_in_ellipse(num_dots, width=1, height=1, center=(0,0)):
    angles = np.random.rand(num_dots)*2*np.pi
    radii = np.sqrt(np.random.rand(num_dots))
    x=center[0]+(width/2)*radii*np.cos(angles)
    y = center[1]+(height/2)*radii*np.sin(angles)
    return x,y

In [6]:
def generate_dots_in_circle(num_dots):
    angles = np.random.rand(num_dots)*2*np.pi
    radii = np.sqrt(np.random.rand(num_dots))
    x=0.5*radii*np.cos(angles)
    y=0.5*radii*np.sin(angles)
    return x,y

In [7]:
def display_dots(num_dots_left,num_dots_right,display_time=0.75,ellipse_scale=1.5):
    ellipse_width = 1*ellipse_scale
    ellipse_height = 1*ellipse_scale
    fig = plt.figure(figsize=(6,3))
    #for generate the dots coordination in left ellipse
    x_left,y_left=generate_dots_in_ellipse(num_dots_left,ellipse_width,ellipse_height)
    #generate the left dots, minus given offset to achieve left coordination
    plt.scatter(x_left-0.75*ellipse_scale,y_left,s=dot_size,c=blue_color)
    #for generate the gots coordination in right ellipse
    x_right,y_right= generate_dots_in_ellipse(num_dots_right,ellipse_width,ellipse_height)
    plt.scatter(x_right+0.75*ellipse_scale,y_right,s=dot_size,c=yellow_color)
    #Ellipse used to draw the ellipse, seoerated to left and right
    ellipse_left = Ellipse((-0.75*ellipse_scale,0),ellipse_width,ellipse_height,edgecolor=black_color,facecolor='none',lw=2)
    ellipse_right = Ellipse((0.75*ellipse_scale,0),ellipse_width,ellipse_height,edgecolor=black_color,facecolor='none',lw=2)
    plt.gca().add_patch(ellipse_left)
    plt.gca().add_patch(ellipse_right)

    plt.xlim(-1.5*ellipse_scale,1.5*ellipse_scale)
    plt.ylim(-0.5*ellipse_scale,0.5*ellipse_scale)
    plt.axis('off')

    plt.show()
    # plt.close(fig)
    #pulse a time for the participants able to see the dots
    time.sleep(display_time)
    # display.clear_output(wait=True)
    #clear.output prepare for next round 
    clear_output()
    time.sleep(0.5)

In [8]:
def instruction_ANS_test():
    global user_id, user_gender, user_hoursleep 
    print("""This is an ANS test to test your number estimati skill. 
             Try you best to recognized which side ellipse contain more dots.
             The time will be recorded and try best to give a correct answer.
             There will be four level in this test and each test contain 10 question.
             PLEASE give some patients to the question come out then input your answer.
             Good luck and enjoy:)""")
    print("Please read:")
    print("")
    print("we wish to record your response data")
    print("to an anonymised public data repository. ")
    print("Your data will be used for educational teaching purposes")
    print("practising data analysis and visualisation.")
    print("")
    print("Please type   yes   in the box below if you consent to the upload.")
    result = input("> ")
    if result == "yes":
        print("Thanks - your data will be uploaded.")
        #send_to_google_form(data_dict, form_url)
    else:
        # end code execution by raising an exception
        raise(Exception("User did not consent to continue test."))

    clear_output(wait=True)
    
    
    id_instructions = """

    Enter your anonymised ID

    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress

    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
         then your unique identifier would be CBTC
    """

    print(id_instructions)
    time.sleep(3)
    user_id = input("> ")  # Now this assigns to the global variable
    clear_output(wait=True)


    gender_instructions = """

    Enter your gender from: "male", "female", or " have not to say"

    
    """

    print(gender_instructions)
    
    time.sleep(3)
    user_gender = input("> ")
    clear_output(wait=True)
    
    
    hoursleep_instructions = """

    Enter how many hours you sleep last night 
    
    """

    print(hoursleep_instructions)
    
    time.sleep(3)
    user_hoursleep = input("> ")
    clear_output(wait=True)
    
    start_enter = """
    
    press enter to start the test
    
    """
    
    print(start_enter)
    
    time.sleep(3)
    start_enter = input(">")
    clear_output(wait=True)
    
    



In [9]:
def run_ANS_test(num_trials_per_level=16):
    global user_id, user_gender, user_hoursleep
    np.random.seed(1)
    
    responses = []
    answers = []
    correct_answers = []
    
    total_time = 0  # Track total time
    correct_responses = 0
    level = 1
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfjd12tCic-30j2tahCDBnKLg1H2BogrJluGV9dEZAOHrAlMQ/viewform?usp=sf_link"

    # Define the possible ratios and corresponding images for each level
    ratios_by_level = {
        1: 5,
        2: 4,
        3: 3
    }

    possible_images = {
        1: [(12, 9), (16, 12), (15, 20), (12, 14), (18, 21)],
        2: [(18, 16)],
        3: [(10, 9), (20, 18)]
    }

    level = 1
    correct_responses = 0
    total_score = 0
    total_correct_responses = 0
    while level <= 3:
        print(f'Level {level}')
        for i in range(num_trials_per_level):
            try:
                # Randomly choose an image for the selected ratio
                num_dots_left, num_dots_right = possible_images[level][np.random.randint(0, len(possible_images[level]))]

                start_time = time.time()  # Start timing for this question
                display_dots(num_dots_left, num_dots_right)
                display_time = time.time() - start_time  # Calculate display time
                total_time += display_time
                panel = widgets.HBox([btn1, btn2])
                display(panel)
                wait_for_event()
                answer = event_info["description"]
                answers.append(answer)
                
                correct_answer = 'Left' if num_dots_left > num_dots_right else 'Right'

                if answer == correct_answer:
                    # print("Correct!")
                    correct_responses += 1
                    correct_answers.append(answer)

                responses.append({
                    'level': level,
                    'trial': i + 1,
                    'num_left': num_dots_left,
                    'num_right': num_dots_right,
                    'user_answer': answer,
                    'correct': answer == correct_answer,
                    "correct_responses":correct_responses
                })
                # print(responses[i]["correct_responses"])

                time.sleep(1)
                clear_output(wait=True)
                time.sleep(0.5)

            except Exception as e:
                print(f"Error occurred in round {i + 1}: {e}")
                break
                
        if responses != []:
            if correct_responses >= ratios_by_level[level]:
                print(f"Level {level} completed. You answered correctly {correct_responses} out of {num_trials_per_level} times.")
                total_correct_responses += responses[-1]["correct_responses"]
                correct_responses = 0  # Reset correct_responses for the next level
                level += 1
                total_score +=1
                # total_correct_responses += responses[6]
                print(f"total score : {total_score}")
                print(f"total_correct_responses : {total_correct_responses}")
                time.sleep(3)
            else:
                print(f"Stay in Level {level} as the required correct responses haven't been achieved.")
                total_correct_responses += responses[-1]["correct_responses"]
                correct_responses = 0  # Reset correct_responses for the next level
                level += 1
                # total_correct_responses += responses["correct_responses"]
                print(f"total_correct_responses : {total_correct_responses}")
                print(f"total score : {total_score}")
                time.sleep(3)


    print(f"""  Well done! The test completed. 
                Total time taken: {total_time} seconds. 
                Total score you got: {total_score}.
                Total correct response you got: {total_correct_responses}.""")
    data_dict = {
        "User id" : user_id,
        "Score" : total_score, 
        "Total time" : total_time,
        "Correct response" : total_correct_responses,
        "gender" : user_gender,
        "Time sleep" : user_hoursleep,
        "user response" : answers,
        "Correct": correct_answers
    }
                
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfjd12tCic-30j2tahCDBnKLg1H2BogrJluGV9dEZAOHrAlMQ/viewform?usp=sf_link"
    send_to_google_form(data_dict, form_url)
    # return responses
    

In [10]:
instruction = instruction_ANS_test()
test_responses = run_ANS_test()
print(instruction)
print(test_responses)

Level 3 completed. You answered correctly 6 out of 16 times.
total score : 3
total_correct_responses : 37
  Well done! The test completed. 
                Total time taken: 63.35694074630737 seconds. 
                Total score you got: 3.
                Total correct response you got: 37.
None
None
